In [1]:
from scipy import sparse
import random
from pyscipopt import Model, quicksum, Expr
from pyscipopt.scip import Term
import time as t

## construct model:
# max {c'x}
# A * x <= b
#   A: random positive sparse matrix 200 x 300
#   b: random right hand side
#   c: objective vector
#   x: variables

numvars    = 200
numconstr  = 300
A = sparse.rand(numconstr,numvars,density=0.005,format="csr")
b = [random.uniform(1,10) for _ in range(numconstr)]
c = [random.uniform(1,10) for _ in range(numvars)]

## 1. benchmark quicksum approach:
m1 = Model()
m1.setMaximize()
x1 = [m1.addVar(obj=c[i]) for i in range(numvars)] # creating variables
t0 = t.time()
for i in range(numconstr): # generating and adding constraints in a loop
    m1.addCons(quicksum(A[i,j]*x1[j] for j in range(numvars)) <= b[i])
print("1: construction time ",t.time()-t0,"seconds")
m1.optimize()
print("1: Optimum ",m1.getObjVal())
m1.freeTransform()
m1.optimize()

## 2. Add rows and set coefficients
m2 = Model()
m2.setMaximize()
x3 = [m2.addVar(obj=c[i]) for i in range(numvars)] # creating variables
t0 = t.time()
constr = [m2.addCons(Expr() <= b_i) for b_i in b]
for k,a_ineq in zip(constr,A):
    X = [x3[i] for i in a_ineq.indices]
    for x,a in zip(X,a_ineq.data):
        m2.addConsCoeff(k,x,a)
print("2: construction time ",t.time()-t0,"seconds")
m2.freeTransform()
m2.optimize()
print("2: Optimum ",m2.getObjVal())
m2.freeTransform()
m2.optimize()

## 3. benchmark manual "Term" creation
m3 = Model()
m3.setMaximize()
x3 = [m3.addVar(obj=c[i]) for i in range(numvars)] # i) creating variables
t0 = t.time()
terms = [Term(x3[i]) for i in range(numvars)] # ii) Creating a "Term" for each variable
for i in range(A.shape[0]):
    for a in A[i]:
        # iii) Creating "Expressions" for each constraint manually
        e = Expr({terms[j]:d for j,d in zip(a.indices,a.data)}) 
        m3.addCons(e <= b[i]) # iv) add constraint
print("3: construction time ",t.time()-t0,"seconds")
m3.optimize()
print("3: Optimum ",m3.getObjVal())
m3.freeTransform()
m3.optimize()

1: construction time  1.731799840927124 seconds
1: Optimum  12687630.425787345
feasible solution found by trivial heuristic after 0.0 seconds, objective value 1.268763e+07
presolving:
problem infeasible or unbounded: variable <t_x196> with objective -1.02727561924369 can be made infinitely large
presolving (1 rounds: 1 fast, 0 medium, 0 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
transformed 2/2 original solutions to the transformed problem space
presolving detected unboundedness
Presolving Time: 0.00

SCIP Status        : problem is solved [unbounded]
Solving Time (sec) : 0.00
Solving Nodes      : 0
Primal Bound       : +1.00000000000000e+20 (2 solutions)
Dual Bound         : +1.00000000000000e+20
Gap                : 0.00 %
2/2 feasible solutions given by solution candidate storage, new primal bound 1.268763e+07

presolving:
problem infeasible or unbound

In [1]:
%load_ext autoreload
from scipy import sparse
import numpy as np
from importlib import reload
import mcs

In [2]:
# example setup (default: minimize)
# main problem
c = [-1,0,0,0]
A_ineq = sparse.csr_matrix([[1,1,0,0],[-1,0,0,0]])
b_ineq = [3,-1]
A_eq = sparse.csr_matrix([[0,1,0,0]])
b_eq = [0.6]
lb = [1,0,0,0]
ub = [np.inf, np.inf, 1, 1]
vtype = 'CCBB'
# indicator constraints
ic_binv = [2,3]
ic_A = sparse.csr_matrix([[1,1,0,0],[1,0,0,0]])
ic_b = [0,1.2]
ic_sense = "LE"
ic_indicval = [1,0]
indic_constr = mcs.Indicator_constraints(ic_binv, ic_A, ic_b, ic_sense, ic_indicval)

In [8]:
%autoreload
my_prob = mcs.MILP_LP(  c=c,
                        A_ineq=A_ineq,
                        b_ineq=b_ineq,
                        A_eq=A_eq,
                        b_eq=b_eq,
                        lb=lb,
                        ub=ub,
                        vtype=vtype, 
                        indic_constr=indic_constr,
                        solver='scip')
x, opt_cx, status = my_prob.solve()
# solution.get_status() returns an integer code
print("Solution status = ", str(status))
# the following line prints the corresponding string
print("Solution value  = ", str(round(opt_cx,5)))
sol = {'x'+str(i) : round(x[i],5) for i in range(len(x))}
print(sol)

Solution status =  0
Solution value  =  -2.4
{'x0': 2.4, 'x1': 0.6, 'x2': 0, 'x3': 1}


In [9]:
%load_ext autoreload
import cobra
from importlib import reload
ecc2 = cobra.io.read_sbml_model("ECC2comp.sbml")
iml1515 = cobra.io.read_sbml_model("iML1515.sbml")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
%autoreload
import mcs
from mcs import fva,fba
import cProfile
reload(mcs)
# opt = ecc2.optimize()
# print(opt)
# cProfile.runctx("fba(iml1515,solver='scip')",globals(),locals())
# opt2 = fba(iml1515,solver='scip')
# print(opt2)
# opt3 = cobra.flux_analysis.variability.flux_variability_analysis(iml1515)
# print(opt3)
# cProfile.runctx("mcs.fva(iml1515)",globals(),locals())
opt4 = fva(iml1515, solver='scip')
print(opt4)
# cProfile.runctx()

           minimum     maximum
CYTDK2    0.000000  228.140000
XPPT      0.000000  114.070000
HXPRT     0.000000  114.070000
NDPK5   -76.922791  114.070000
SHK3Dr    0.000000    5.635459
...            ...         ...
MPTS      0.000000   -0.000000
MOCOS     0.000000   -0.000000
BMOGDS2   0.000000   -0.000000
FESD2s    0.000000   -0.000000
OCTNLL    0.000000   -0.000000

[2712 rows x 2 columns]
